In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip3 install transformers
!pip3 install simpletransformers
import pandas as pd
import logging
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch
import dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/Electra_Trained.zip" -d "/content/electra"
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/MedElectra_Trained.zip" -d "/content/medelectra"
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/Roberta_Trained.zip" -d "/content/roberta"
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/biobert_2_epch.zip" -d "/content/biobert"
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/saved_model_distil_ro_berta.zip" -d "/content/distilled_roberta"

Archive:  /content/drive/MyDrive/472 Project Files/Trained Models/Electra_Trained.zip
   creating: /content/electra/content/outputs/checkpoint-58284-epoch-4/
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/training_args.bin  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/config.json  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/pytorch_model.bin  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/tokenizer_config.json  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/scheduler.pt  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/vocab.txt  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/optimizer.pt  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/model_args.json  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/special_tokens_map.json  
  inflating: /content/electra/content/o

In [ ]:
import os

def get_model_folder(model_name, base_folder="/content/"):
  location = f"{base_folder}{model_name}/content/outputs"
  checkpoint_folder = os.listdir(location)[0]
  return f"{location}/{checkpoint_folder}"

In [ ]:
model_args = ClassificationArgs(num_train_epochs=4)
kwargs = {"num_labels": 40, "args": model_args, 
            "use_cuda": torch.cuda.is_available()}

modelElectra = ClassificationModel("electra", get_model_folder("electra"), **kwargs)
modelMedElectra = ClassificationModel("electra", get_model_folder("medelectra"),**kwargs)
modelRoBERTa = ClassificationModel("roberta", get_model_folder("roberta"), **kwargs)
modelDistilledRoBERTa = ClassificationModel("roberta", get_model_folder("distilled_roberta"), **kwargs)
modelBioBERT = ClassificationModel("bert", get_model_folder("biobert"), **kwargs)

In [ ]:
models = [modelElectra, modelMedElectra, modelRoBERTa, modelDistilledRoBERTa, modelBioBERT]
names = ["electra", "medelectra", "roberta", "distilled_roberta", "biobert"]

In [ ]:
def dataframe_from_file(filename, 
    base = "/content/drive/MyDrive/472 Project Files/Stratified Input Data/"):
  
  df = pd.read_csv(base + filename)
  # Remove NaNs
  are_text_strings = df.applymap(lambda x: isinstance(x, (str)))['text']
  df = df[are_text_strings]
  # Fix label
  df["labels"] = df["label"]
  return df

In [ ]:
eval_df = dataframe_from_file("eval_df-3.csv")
result = modelElectra.eval_model(eval_df)

  0%|          | 0/38850 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/4857 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, wrong_prediction = result

In [ ]:
eval_df = dataframe_from_file("eval_df-3.csv")
for model, name in zip(models, names):
    result = model.eval_model(eval_df)
    file_name = f'/content/eval_results/eval_{name}.pkl'
    with open(file_name, 'wb') as file:
        dill.dump(result, file)
        print(f'Object successfully saved to "{file_name}"')

  0%|          | 0/38850 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/4857 [00:00<?, ?it/s]

Object successfully saved to "/content/eval_results/eval_electra.pkl"


  0%|          | 0/38850 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/4857 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/4857 [00:00<?, ?it/s]

Object successfully saved to "/content/eval_results/eval_distilled_roberta.pkl"


  0%|          | 0/38850 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/4857 [00:00<?, ?it/s]

Object successfully saved to "/content/eval_results/eval_biobert.pkl"


In [ ]:
!mv "/content/eval_results" "/content/drive/MyDrive/472 Project Files/eval_results"